DATTABAYÖ - Borja Esteve Molner & Jorge López Fresco

## Sesión 2/5 - Práctica 3 - LNR
### Clasificación automática

___________
**NOTA IMPORTANTE**: **Te hemos reenviado la Sesión 1 de la práctica 3 en la entrega de la Sesión 2. En ella hemos realizado cambios y mejorado las representaciones de palabras.**
__________

**Objetivo**: Implementar un clasificador para los datos de la tarea de detección de estereotipos en DETESTS. Es importante que se entren al menos dos clasificadores diferentes y se comparen sus resultados. Entregar código e informe con la explicación de la propuesta antes de la siguiente sesión. 

**Importar Librerías**

In [12]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from imblearn.combine import SMOTETomek
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Para llevar a cabo esta segunda tarea, vamos a trabajar con las 3 representaciones que construimos durante la sesión 1 de esta misma práctica. Dichas representaciones fueron exportadas en numpy mediante la función `np.savetxt()` con el fin de facilitar el posterior trabajo. Estas representaciones representarán a la matriz `X` en los modelos de clasificación automática. 

Cargaremos las siguientes representaciones de palabras:  

In [4]:
ruta = "/content/drive/MyDrive/3º - GCD/2C - LENGUAJE NATURAL Y RECUPERACIÓN DE LA INFORMACIÓN/Prácticas LNR/Práctica 3 (Sesiones 1 a 5) - LNR/Práctica 3 - Sesión 1 y 2 - LNR/"

# Representación con Word Embedding - Ponderada
data_pond = np.loadtxt(ruta+"rep_pond_w_emb.txt")
# Representación con Word Embedding - Mediana
data_med = np.loadtxt(ruta+"rep_median_w_emb.txt")
# Representación con 3-gramas
data_ngram = np.loadtxt(ruta+"represent_ngramas.txt")

Además de la matriz `X`, necesitaremos un vector de etiquetas `Y` que contendrá valores binarios de la pertenencia o no a la clase "stereotype". Para ello cargaremos el dataset original de Training `train.csv`.

In [5]:
df_detests = pd.read_csv(ruta+"train.csv")
extra_Data = pd.DataFrame(df_detests['stereotype']).to_numpy()

## Representación con Word Embedding - Ponderada

El primer paso es construir la matriz de trabajo uniendo el vector de estereotipos con la primera de las representaciones utilizadas. Una vez hecho esto, creamos las matrices `X` e `y`. Separamos los datos en "train" y "eval" para poder entrenar al modelo y evaluarlo posteriormente, con el fin de obtener una primera aproximación de la precisión de la clasificación.

In [6]:
data1 = np.append(data_pond, extra_Data, axis=1)
X, y = data1[0:3817,0:300], data1[0:3817,300:301]
print(data_pond.shape, data1.shape)

(3817, 300) (3817, 301)


In [7]:
x_train, x_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
clf1 = svm.SVC( kernel='rbf', C=2, gamma='scale' ).fit( x_train, y_train )
pred1 = clf1.predict(x_eval)
acc_rbf_svm1 = clf1.score(x_eval, y_eval)
pred_sc1 = pred1
print("Training set score: %f" % clf1.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_rbf_svm1)
print("F1 Score:", f1_score(y_eval, pred1, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc1))
print(classification_report(y_eval, pred_sc1))

Training set score: 0.881318
Accuracy Radial Basis Kernel: 0.8289703315881326
F1 Score: 0.6996357373243542

 [[851  45]
 [151  99]]
              precision    recall  f1-score   support

         0.0       0.85      0.95      0.90       896
         1.0       0.69      0.40      0.50       250

    accuracy                           0.83      1146
   macro avg       0.77      0.67      0.70      1146
weighted avg       0.81      0.83      0.81      1146



Tanto el "Training set score" como el "Accuracy" tienen valores muy aceptables. No obstante, si observamos con detalle la matriz de confusión, podemos ver como los valores de "precision", "recall" y "f1-score" existe una descompensación entre la predicción de los valores etiquetados como 0 y los etiquetados como 1. Esto es debido a que **los datos no están balanceados**. 

Para solucionar el problema de desbalanceo que sufre la muestra, hemos probado dos opciones:

- Primero probamos a penalizar la clase mayoritaria, cambiando el parámetro `class_weight` al valor `balanced`. No obstante, los resultados no eran muy precisos y no parecía solucionar el problema de los datos desbalanceados.

- Como segunda opción, usamos una **técnica de resampling** (existen diversas técnicas basadas en oversampling, undersampling...). En nuestro caso, seleccionamos un método combinado ya que pensamos que ofrecería mejores resultados, principalmente porque el método `SMOTe`, en numerosas ocaiones, genera nuevas muestras ruidosas al generar nuevos puntos entre los valores atípicos y los valores internos. La solución consiste en limpiar el espacio resultante del sobremuestro, es decir, hacer un submuestreo. Así, finalmente decidimos usar el método `SMOTE tomek`.

In [9]:
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
x_train, x_eval, y_train, y_eval = train_test_split(X_resampled, y_resampled, 
                                                    test_size=0.3, random_state=42)

A continuación, presentaremos los modelos que hemos escogido para entrenar con cada representación de palabras. Así, para **elegir los mejores modelos** nos hemos basado en 3 aspectos fundamentales:

1. El valor del accuracy en la clase target (stereotype). 
2. No buscamos un accuracy alto del modelo en general, sino que priorizamos que el accuracy sea alto en la detección de estereotipos.
3. Un f1 score alto, que hace patente la existencia de un buen ratio entre el accuracy (la precisión) y el recall (la cantidad de estereotipos que somos capaces de identificar).

#### Modelo #1 - Ponderada
SVM, Kernel Rbf, C=7, Gamma=Scale

In [11]:
clf4 = svm.SVC(kernel='rbf',gamma = 'scale', C=7).fit( x_train, y_train )
pred4 = clf4.predict(x_eval)
acc_sig_svm4 = clf4.score(x_eval, y_eval)
pred_sc4 = pred4
print("Training set score: %f" % clf4.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_sig_svm4)
print("F1 Score:", f1_score(y_eval, pred4, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc4))
print(classification_report(y_eval, pred_sc4) )

Training set score: 0.951225
Accuracy Radial Basis Kernel: 0.8862478777589134
F1 Score: 0.8861773010938186

 [[761 133]
 [ 68 805]]
              precision    recall  f1-score   support

         0.0       0.92      0.85      0.88       894
         1.0       0.86      0.92      0.89       873

    accuracy                           0.89      1767
   macro avg       0.89      0.89      0.89      1767
weighted avg       0.89      0.89      0.89      1767



Viendo la matriz de confusión se puede observar una clara mejoría tanto en el "Training set score" como en el "Accuracy" o el "f1-score". Así mismo, los valores de "precision", "recall" y "f1-score" para la predicción muestran valores mucho más cercanos y aceptables que los obtenidos anteriormente. 

#### Modelo #2 - Ponderada
MLP, 3 capas ocultas, solver=adam, activación=sigmoide.

In [14]:
mlp = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=1000, alpha=1e-4,
                    solver='adam', activation='logistic', random_state=1,
                    learning_rate_init=.1)
mlp.fit(x_train, y_train)
print("Training set score: %f" % mlp.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', mlp.score(x_eval, y_eval))
print("F1 Score:", f1_score(y_eval, mlp.predict(x_eval), average='macro'))
print("\n", confusion_matrix(y_eval, mlp.predict(x_eval)))
print(classification_report(y_eval, mlp.predict(x_eval)))

Training set score: 0.972579
Accuracy Radial Basis Kernel: 0.8698358800226372
F1 Score: 0.8697787831793189

 [[750 144]
 [ 86 787]]
              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87       894
         1.0       0.85      0.90      0.87       873

    accuracy                           0.87      1767
   macro avg       0.87      0.87      0.87      1767
weighted avg       0.87      0.87      0.87      1767



#### Modelo #3 - Ponderada
SVM, Kernel Rbf, C=2, Gamma=Scale

In [15]:
clf1 = svm.SVC( kernel='rbf', C=2, gamma='scale' ).fit( x_train, y_train )
pred1 = clf1.predict(x_eval)
acc_rbf_svm1 = clf1.score(x_eval, y_eval)
pred_sc1 = pred1
print("Training set score: %f" % clf1.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_rbf_svm1)
print("F1 Score:", f1_score(y_eval, pred1, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc1))
print(classification_report(y_eval, pred_sc1))

Training set score: 0.908517
Accuracy Radial Basis Kernel: 0.8670062252405206
F1 Score: 0.8669569672295285

 [[749 145]
 [ 90 783]]
              precision    recall  f1-score   support

         0.0       0.89      0.84      0.86       894
         1.0       0.84      0.90      0.87       873

    accuracy                           0.87      1767
   macro avg       0.87      0.87      0.87      1767
weighted avg       0.87      0.87      0.87      1767



________________

**Ganador - Word Embedding Ponderado**

El modelo con SVM, kernel Radial, c=7 y gamma="scale". 

Nos ofrece el accuracy y el f1-score más alto en los 3 casos. De hecho, es el más alto de entre todos los modelos entrenados. Además, los valores de precision, recall y f1-score para la predicción son muy cercanos y elevados para ambas clases (no estereotipo y estereotipo). En general, la mejor predicción de todas. 

               precision    recall  f1-score   support

         0.0       0.92      0.85      0.88       894
         1.0       0.86      0.92      0.89       873
____________________

## Representación con Word Embedding - Mediana

Realizamos, de nuevo, el mismo proceso que hemos seguido con la representación ponderada, pero en este caso usando los datos de la representación con la mediana. También balanceamos los datos para obtener un modelo mejor.

In [16]:
data2 = np.append(data_med, extra_Data, axis=1)
X, y = data2[0:3817,0:300], data2[0:3817,300:301]
print(data_med.shape, data2.shape)

(3817, 300) (3817, 301)


In [17]:
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
x_train, x_eval, y_train, y_eval = train_test_split(X_resampled, y_resampled, 
                                                    test_size=0.3, random_state=42)

#### Modelo #1 - Mediana
SVM, Kernel Polinómico grado 4, C=8

In [22]:
clf3 = svm.SVC(kernel='poly', C=8, degree=4).fit( x_train, y_train )
pred3 = clf3.predict(x_eval)
acc_pol_svm3 = clf3.score(x_eval, y_eval)
pred_sc3 = pred3
print("Training set score: %f" % clf3.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_pol_svm3)
print("F1 Score:", f1_score(y_eval, pred3, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc3))
print(classification_report(y_eval, pred_sc3))

Training set score: 0.953641
Accuracy Radial Basis Kernel: 0.8527746319365799
F1 Score: 0.8515561452445334

 [[673 203]
 [ 57 833]]
              precision    recall  f1-score   support

         0.0       0.92      0.77      0.84       876
         1.0       0.80      0.94      0.87       890

    accuracy                           0.85      1766
   macro avg       0.86      0.85      0.85      1766
weighted avg       0.86      0.85      0.85      1766



#### Modelo #2 - Mediana
MLP, 3 capas ocultas, solver=adam, activation=sigmoide

In [21]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,10,10), max_iter=1000, alpha=1e-4,
                    solver='adam', activation='logistic', random_state=1,
                    learning_rate_init=.1)
mlp2.fit(x_train, y_train)
print("Training set score: %f" % mlp2.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', mlp2.score(x_eval, y_eval))
print("F1 Score:", f1_score(y_eval, mlp2.predict(x_eval), average='macro'))
print("\n", confusion_matrix(y_eval, mlp2.predict(x_eval)))
print(classification_report(y_eval, mlp2.predict(x_eval)))

Training set score: 0.962621
Accuracy Radial Basis Kernel: 0.8431483578708947
F1 Score: 0.8426737189512781

 [[696 180]
 [ 97 793]]
              precision    recall  f1-score   support

         0.0       0.88      0.79      0.83       876
         1.0       0.82      0.89      0.85       890

    accuracy                           0.84      1766
   macro avg       0.85      0.84      0.84      1766
weighted avg       0.85      0.84      0.84      1766



#### Modelo #3 - Mediana
SVM, Kernel Rbf, C=15, Gamma=Scale

In [26]:
clf2 = svm.SVC( kernel='rbf', C=15, gamma='scale' ).fit( x_train, y_train )
pred2 = clf1.predict(x_eval)
acc_rbf_svm2 = clf2.score(x_eval, y_eval)
pred_sc2 = pred2
print("Training set score: %f" % clf2.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_rbf_svm2)
print("F1 Score:", f1_score(y_eval, pred2, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc2))
print(classification_report(y_eval, pred_sc2))

Training set score: 0.965534
Accuracy Radial Basis Kernel: 0.8646659116647791
F1 Score: 0.6736774600276084

 [[820  56]
 [488 402]]
              precision    recall  f1-score   support

         0.0       0.63      0.94      0.75       876
         1.0       0.88      0.45      0.60       890

    accuracy                           0.69      1766
   macro avg       0.75      0.69      0.67      1766
weighted avg       0.75      0.69      0.67      1766



_____________

**Ganador - Word Embedding con Mediana**

El modelo con SVM, kernel Polinómico grado 4 y C=8. 

Nos ofrece un accuracy y el f1-score aceptablemente bueno. Además, los valores de precision, recall y f1-score para la predicción son muy cercanos y elevados para ambas clases (no estereotipo y estereotipo). Es un buen modelo. 

               precision    recall  f1-score   support

         0.0       0.92      0.77      0.84       876
         1.0       0.80      0.94      0.87       890
_____________

## Representación con 3-Gramas

Por último, tenemos la representación mediante 3-gramas. De nuevo, realizaremos el mismo proceso que hemos seguido con las representaciones anteriores. No obstante, en este caso la cantidad de datos representados es distinta y el coste computacional mucho mayor (casi 1 hora de ejecución) que en los casos anteriores. Por este motivo, para los 3-gramas solo presentaremos 1 modelo.

In [27]:
data3 = np.append(data_ngram, extra_Data, axis=1)
X, y = data3[0:3817,0:42766], data3[0:3817,42766:42767]
print(data_ngram.shape, data3.shape)

(3817, 42766) (3817, 42767)


In [28]:
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
x_train, x_eval, y_train, y_eval = train_test_split(X_resampled, y_resampled, 
                                                    test_size=0.3, random_state=42)

In [29]:
print(x_train.shape, y_train.shape)

(4124, 42766) (4124,)


#### Modelo #1 - 3_gramas
SVM, Kernel Rbf, C2, Gamma Scale

In [31]:
clf5 = svm.SVC( kernel='rbf', C=2, gamma='scale' ).fit( x_train, y_train )
pred5 = clf5.predict(x_eval)
acc_rbf_svm5 = clf5.score(x_eval, y_eval)
pred_sc5 = pred5
print("Training set score: %f" % clf5.score( x_train, y_train ) )
print('Accuracy Radial Basis Kernel:', acc_rbf_svm5)
print("F1 Score:", f1_score(y_eval, pred5, average='macro'))
print("\n", confusion_matrix(y_eval, pred_sc5))
print(classification_report(y_eval, pred_sc5))

Training set score: 0.952473
Accuracy Radial Basis Kernel: 0.8110859728506787
F1 Score: 0.8106983853429351

 [[677 234]
 [100 757]]
              precision    recall  f1-score   support

         0.0       0.87      0.74      0.80       911
         1.0       0.76      0.88      0.82       857

    accuracy                           0.81      1768
   macro avg       0.82      0.81      0.81      1768
weighted avg       0.82      0.81      0.81      1768



_____________

**Ganador - 3_gramas**

El modelo con SVM, kernel rbf, C=2 y Gamma=scale. 

Nos ofrece un accuracy y el f1-score aceptable. Por su parte, los valores de precision, recall y f1-score para la predicción son cercanos y aceptables para ambas clases (no estereotipo y estereotipo). Es un modelo aceptable. 

               precision    recall  f1-score   support

         0.0       0.87      0.74      0.80       911
         1.0       0.76      0.88      0.82       857
_____________

## Comparativa final de Modelos Ganadores

Estos son nuestros 3 modelos elegidos, uno por cada tipo de representación:

**Modelo para Word Embedding Ponderado**

El modelo con SVM, kernel Radial, c=7 y gamma="scale". 

- Training set score: 0.951225
- Accuracy Radial Basis Kernel: 0.8862478777589134
- F1 Score: 0.8861773010938186

              precision    recall  f1-score   support

         0.0       0.92      0.85      0.88       894
         1.0       0.86      0.92      0.89       873

**Modelo para Word Embedding con Mediana**

El modelo con SVM, kernel Polinómico grado 4 y C=8. 

- Training set score: 0.953641
- Accuracy Radial Basis Kernel: 0.8527746319365799
- F1 Score: 0.8515561452445334

              precision    recall  f1-score   support

         0.0       0.92      0.77      0.84       876
         1.0       0.80      0.94      0.87       890
    
**Modelo para 3_gramas**

El modelo con SVM, kernel rbf, C=2 y Gamma=scale. 

- Training set score: 0.952473
- Accuracy Radial Basis Kernel: 0.8110859728506787
- F1 Score: 0.8106983853429351

              precision    recall  f1-score   support

         0.0       0.87      0.74      0.80       911
         1.0       0.76      0.88      0.82       857